In [2]:
#|export
import pandas as pd 
import numpy as np 
import plotly 
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly_express as px
import streamlit as st
import altair as alt

st.set_page_config(
    page_title="World travel Data",
    page_icon="🌍",
    layout="wide",
    initial_sidebar_state="expanded")



pd.set_option("display.float.format", lambda x: "%.2f" % x)

2025-02-23 11:46:06.070 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:
#|export
inbound_data = pd.read_excel('travel_data.xls', sheet_name="number_of_arrival", header=3, index_col=0)
#inbound_data.head()

In [4]:
#|export
country_codes = pd.read_excel('travel_data.xls', sheet_name= 'country code')
#country_codes.head()

In [5]:
#|export
inbound_data = pd.merge(country_codes, inbound_data, left_on = "Country Code", right_on = "Country Code", how="left")
#inbound_data.head()

In [6]:
#|export
inbound_data = inbound_data.set_index("Country Name")
#inbound_data.head()

In [7]:
#|export
#select columns from year 2007 to 2017
new_inbound_data = pd.concat([inbound_data["Country Code"],inbound_data.loc[:,"2005":"2017"]], axis=1)
#new_inbound_data.head()

In [8]:
#|export
#replace missing value by 0
new_inbound_data = new_inbound_data.fillna(0)
#new_inbound_data.head()

In [9]:
#|export
new_inbound_data.set_index(["Country Code"], inplace = True, append = True)
#new_inbound_data.head()

In [10]:
#|export
#use stack to change the data frame shape
new_inbound_data = new_inbound_data.stack()
#new_inbound_data.head()

In [11]:
#|export
new_inbound_data = pd.DataFrame(new_inbound_data)
#new_inbound_data.head()

In [12]:
#|export
new_inbound_data = new_inbound_data.reset_index()
#new_inbound_data.head()

In [13]:
#|export
#rename column names
new_inbound_data= new_inbound_data.rename(columns = {"level_2":"Year",0:"Amount"})
#new_inbound_data.head()

In [14]:

inbound_map = px.choropleth(new_inbound_data, 
                            locations="Country Code", 
                            color="Amount", 
                            hover_name="Country Name",
                            #set the animation slide bar
                            animation_frame="Year",
                            color_continuous_scale=px.colors.sequential.BuGn,
                            #set the map type
                            projection="natural earth",
                            width = 2000, 
                            height= 1000)
inbound_map.show(renderer='iframe')

In [15]:
#|export
#find the top countries that attract tourists
top_inbound = inbound_data.sort_values("2017", ascending=False).head(10)
top_inbound_list = list(top_inbound.index)



# In[11]:


#select the top countries' data 
top_inbound_data = new_inbound_data.loc[new_inbound_data["Country Name"].isin(top_inbound_list)]
#top_inbound_data.head()

In [16]:

inbound_top = px.line(top_inbound_data, 
                      x="Year", 
                      y="Amount", 
                      color="Country Name", 
                      line_group="Country Name",  
                      line_shape="linear", 
                      title="Top countries that attract tourists")
inbound_top.show(renderer='iframe')

In [17]:
#|export
#importing all data  

inbound = pd.read_excel('travel_data.xls', sheet_name="number_of_arrival", header=3, index_col=0)
outbound = pd.read_excel('travel_data.xls', sheet_name="number_of_departure", header=3, index_col=0)
population = pd.read_excel('travel_data.xls', sheet_name="population", header=3, index_col=0)
GDP = pd.read_excel('travel_data.xls', sheet_name="GDP", header=3, index_col=0)
country_names = pd.read_excel('travel_data.xls', sheet_name= 'country code')



In [18]:
#|export
#select only the relevant columns 2005 to 2017
def select_data(df):
    amount = df['Indicator Name'][0]
    df = pd.merge(country_names, df, left_on = 'Country Code', right_on = 'Country Code', how = 'left')
    df = df.set_index('Country Name')
    df = pd.concat([df['Country Code'],df.loc[:,'2005':'2017']], axis=1)
    df = df.fillna(0)
    df = df.set_index(["Country Code"], append = True)
    df = df.stack()
    df = pd.DataFrame(df)
    df = df.reset_index()
    df = df.rename(columns = {"level_2":"Year", 0:amount})
    return df
inbound = select_data(inbound)
#inbound.head()
    
                              

/var/folders/4y/67_mnd3d1qj8_bphj56jl1200000gn/T/ipykernel_91553/1022752012.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [19]:
#|export
outbound = select_data(outbound)
#outbound.head()

/var/folders/4y/67_mnd3d1qj8_bphj56jl1200000gn/T/ipykernel_91553/1022752012.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [20]:
#|export
population = select_data(population)
#population.head()

/var/folders/4y/67_mnd3d1qj8_bphj56jl1200000gn/T/ipykernel_91553/1022752012.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [21]:
#|export
GDP = select_data(GDP)
#GDP.head()

/var/folders/4y/67_mnd3d1qj8_bphj56jl1200000gn/T/ipykernel_91553/1022752012.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [22]:
#|export
complete_data = inbound.merge(outbound)
#complete_data.head()

In [23]:
#|export
complete_data = complete_data.merge(population) 
#complete_data.head()

In [24]:
#|export
complete_data = complete_data.merge(GDP)
#complete_data.head()

In [25]:
#|export
complete_data.Year = complete_data.Year.astype('int64')

In [26]:
#|export

alt.theme.enable("dark")

ThemeRegistry.enable('dark')

In [27]:
#|export
with st.sidebar:
    st.title('World travel Dashboard')
    
    year_list = list(complete_data.Year.unique())[::-1]
    
    selected_year = st.selectbox('Select a year', year_list, index=len(year_list)-1)
    df_selected_year = complete_data[complete_data.Year == selected_year]
    df_selected_year_sorted = df_selected_year.sort_values(by="International tourism, number of arrivals", ascending=False)

    color_theme_list = ['blues', 'cividis', 'greens', 'inferno', 'magma', 'plasma', 'reds', 'rainbow', 'turbo', 'viridis']
    selected_color_theme = st.selectbox('Select a color theme', color_theme_list)

2025-02-23 11:46:06.651 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.763 
  command:

    streamlit run /Users/borja/anaconda3/envs/Kaggle/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-02-23 11:46:06.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.765 Se

In [28]:
#|export
def make_heatmap(input_df, input_y, input_x, input_color, input_color_theme):
    heatmap = alt.Chart(input_df).mark_rect().encode(
            y=alt.Y(f'{input_y}:O', axis=alt.Axis(title="Year", titleFontSize=18, titlePadding=15, titleFontWeight=900, labelAngle=0)),
            x=alt.X(f'{input_x}:O', axis=alt.Axis(title="", titleFontSize=18, titlePadding=15, titleFontWeight=900)),
            color=alt.Color(f'max({input_color}):Q',
                             legend=None,
                             scale=alt.Scale(scheme=input_color_theme)),
            stroke=alt.value('black'),
            strokeWidth=alt.value(0.25),
        ).properties(width=900
        ).configure_axis(
        labelFontSize=12,
        titleFontSize=12
        ) 
    # height=300
    return heatmap

In [29]:
#|export
def make_choropleth(input_df, input_id, input_column, input_color_theme):
    choropleth = px.choropleth(input_df, locations=input_id, color=input_column, locationmode="country names",
                               color_continuous_scale=input_color_theme,
                               range_color=(0, max(df_selected_year['International tourism, number of arrivals'])),
                               scope="world",
                               labels={'International tourism, number of arrivals':'International arrivals'}
                              )
    choropleth.update_layout(
        template='plotly_dark',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=0, b=0),
        height=350
    )
    return choropleth

In [30]:
#|export
def make_line(input_df, input_x,input_y):
    line = px.line(input_df, 
                      x=input_x, 
                      y=input_y, 
                      color= "Country Name", 
                      line_group="Country Name",  
                      line_shape="linear"
                      )
    line.update_layout(
        template='plotly_dark',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=0, b=0),
        height=350
    )
    return line

In [31]:
#|export
def calculate_arrivals_difference(input_df, input_year):
  selected_year_data = input_df[input_df['Year'] == input_year].reset_index()
  previous_year_data = input_df[input_df['Year'] == input_year - 1].reset_index()
  selected_year_data['incoming_difference'] = selected_year_data['International tourism, number of arrivals'].sub(previous_year_data['International tourism, number of arrivals'], fill_value=0)
  return pd.concat([selected_year_data['Country Name'], selected_year_data['Country Code'],selected_year_data['International tourism, number of arrivals'], selected_year_data['incoming_difference']], axis=1).sort_values(by="incoming_difference", ascending=False)

    

In [32]:
#|export
def make_donut(input_response, input_text, input_color):
  if input_color == 'blue':
      chart_color = ['#29b5e8', '#155F7A']
  if input_color == 'green':
      chart_color = ['#27AE60', '#12783D']
  if input_color == 'orange':
      chart_color = ['#F39C12', '#875A12']
  if input_color == 'red':
      chart_color = ['#E74C3C', '#781F16']
    
  source = pd.DataFrame({
      "Topic": ['', input_text],
      "% value": [100-input_response, input_response]
  })
  source_bg = pd.DataFrame({
      "Topic": ['', input_text],
      "% value": [100, 0]
  })
    
  plot = alt.Chart(source).mark_arc(innerRadius=45, cornerRadius=25).encode(
      theta="% value",
      color= alt.Color("Topic:N",
                      scale=alt.Scale(
                          #domain=['A', 'B'],
                          domain=[input_text, ''],
                          # range=['#29b5e8', '#155F7A']),  # 31333F
                          range=chart_color),
                      legend=None),
  ).properties(width=130, height=130)
    
  text = plot.mark_text(align='center', color="#29b5e8", font="Lato", fontSize=32, fontWeight=700, fontStyle="italic").encode(text=alt.value(f'{input_response} %'))
  plot_bg = alt.Chart(source_bg).mark_arc(innerRadius=45, cornerRadius=20).encode(
      theta="% value",
      color= alt.Color("Topic:N",
                      scale=alt.Scale(
                          # domain=['A', 'B'],
                          domain=[input_text, ''],
                          range=chart_color),  # 31333F
                      legend=None),
  ).properties(width=130, height=130)
  return plot_bg + plot + text

In [33]:
#|export
def format_number(num):
    if num > 1000000:
        if not num % 1000000:
            return f'{num // 1000000} M'
        return f'{round(num / 1000000, 1)} M'
    return f'{num // 1000} K'

In [34]:
#|export
col = st.columns((3, 5), gap='medium')

2025-02-23 11:46:06.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.797 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.797 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [35]:
#|export
with col[0]:
    st.markdown('#### Gains/Losses')

    df_arrivals_difference_sorted = calculate_arrivals_difference(complete_data, selected_year)

    if selected_year > 2010:
        first_country_name = df_arrivals_difference_sorted['Country Name'].iloc[0]
        first_country_arrivals = format_number(df_arrivals_difference_sorted['International tourism, number of arrivals'].iloc[0])
        first_country_delta = format_number(df_arrivals_difference_sorted.incoming_difference.iloc[0])
    else:
        first_country_name = '-'
        first_country_arrivals = '-'
        first_country_delta = ''
    st.metric(label=first_country_name, value=first_country_arrivals, delta=first_country_delta)

    if selected_year > 2010:
        last_country_name = df_arrivals_difference_sorted['Country Name'].iloc[-1]
        last_country_arrivals = format_number(df_arrivals_difference_sorted['International tourism, number of arrivals'].iloc[-1])   
        last_country_delta = format_number(df_arrivals_difference_sorted.incoming_difference.iloc[-1])   
    else:
        last_country_name = '-'
        last_country_arrivals = '-'
        last_country_delta = ''
    st.metric(label=last_country_name, value=last_country_arrivals, delta=last_country_delta)

    
    st.markdown('#### Countries Tourism')

    if selected_year > 2010:
        # Filter countries with arrivals difference > 50000
        # df_greater_50000 = df_population_difference_sorted[df_population_difference_sorted.population_difference_absolute > 50000]
        df_greater_50000 = df_arrivals_difference_sorted[df_arrivals_difference_sorted.incoming_difference > 50000]
        df_less_50000 = df_arrivals_difference_sorted[df_arrivals_difference_sorted.incoming_difference < -50000]
        
        # % of Countries with arrivals difference > 50000
        countries_tourism_greater = round((len(df_greater_50000)/df_arrivals_difference_sorted['Country Name'].nunique())*100)
        countries_tourism_less = round((len(df_less_50000)/df_arrivals_difference_sorted['Country Name'].nunique())*100)
        donut_chart_greater = make_donut(countries_tourism_greater, 'Tourism increased', 'green')
        donut_chart_less = make_donut(countries_tourism_less, 'Tourism decreased', 'red')
    else:
        countries_tourism_greater = 0
        countries_tourism_less = 0
        donut_chart_greater = make_donut(countries_tourism_greater, 'Tourism increased', 'green')
        donut_chart_less = make_donut(countries_tourism_less, 'Tourism decreased', 'red')

    tourism_col = st.columns((0.2, 1, 0.2))
    with tourism_col[1]:
        st.write('Increased')
        st.altair_chart(donut_chart_greater)
        st.write('Decreased')
        st.altair_chart(donut_chart_less)

    st.markdown('#### Top Countries')

    st.dataframe(df_selected_year_sorted,
                 column_order=("Country Name", "International tourism, number of arrivals"),
                 hide_index=True,
                 width=None,
                 column_config={
                    "Country Name": st.column_config.TextColumn(
                        "Countries",
                    ),
                    "International tourism, number of arrivals": st.column_config.ProgressColumn(
                        "Tourism arrivals",
                        format="%f",
                        min_value=0,
                        max_value=max(df_selected_year_sorted['International tourism, number of arrivals']),
                     )}
                 )
    
    with st.expander('About', expanded=True):
        st.write('''
            - Data: World Tourism Organization, Yearbook of Tourism Statistics, Compendium of Tourism Statistics and data files.
            - Thanks to Chanin Nantasenamat, for this amazing tutorial: https://blog.streamlit.io/crafting-a-dashboard-app-in-python-using-streamlit/
            - :orange[**Gains/Losses**]: Countries with difference in tourism arrivals for selected year
            - :orange[**Countries tourism**]: percentage of countries with annual inbound/ outbound tourism > 50,000
            '''
                )
        

2025-02-23 11:46:06.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [36]:
#|export
with col[1]:
    st.markdown('#### Total Tourism')
    
    choropleth = make_choropleth(df_selected_year, 'Country Name', 'International tourism, number of arrivals', selected_color_theme)
    st.plotly_chart(choropleth, use_container_width=True)
    
    heatmap = make_heatmap(complete_data, 'Year', 'Country Name', 'International tourism, number of arrivals', selected_color_theme)
    st.altair_chart(heatmap, use_container_width=True)

    st.markdown('#### Top countries that attract tourists')

    line = make_line(top_inbound_data,'Year', 'Amount')
    st.plotly_chart(line, use_container_width=True)

2025-02-23 11:46:06.864 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:06.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:07.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:07.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:07.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:07.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:07.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-23 11:46:07.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [37]:

#with col[2]:
    #st.markdown('#### Top Countries')

    #st.dataframe(df_selected_year_sorted,
                # column_order=("Country Name", "International tourism, number of arrivals"),
                # hide_index=True,
                # width=None,
                # column_config={
                    #"Country Name": st.column_config.TextColumn(
                       # "Countries",
                  #  ),
                   # "International tourism, number of arrivals": st.column_config.ProgressColumn(
                       # "Tourism arrivals",
                       # format="%f",
                       # min_value=0,
                       # max_value=max(df_selected_year_sorted['International tourism, number of arrivals']),
                     #)}
                 #)
    
    #with st.expander('About', expanded=True):
       # st.write('''
           # - Data: World travel data.
           # - :orange[**Gains/Losses**]: Countries with difference in tourism arrivals for selected year
          #  - :orange[**Countries tourism**]: percentage of countries with annual inbound/ outbound tourism > 50,000
           # ''')

In [38]:
import nbdev
notebook_name = "Exploratory data analisys.ipynb"
export_destination = "." # the root directory
nbdev.export.nb_export(notebook_name,export_destination, name = 'app')

In [39]:
! streamlit run app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.33:8501

/Users/borja/Desktop/Desktop - Borja’s Mac mini/Portfolio/EDA/app.py:104: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  amount = df['Indicator Name'][0]
/Users/borja/Desktop/Desktop - Borja’s Mac mini/Portfolio/EDA/app.py:104: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  amount = df['Indicator Name'][0]
/Users/borja/Desktop/Desktop - Borja’s Mac mini/Portfolio/EDA/app.py:104: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys w